In [1]:
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments
import datasets
from torch.utils.data import DataLoader

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048
dtype=None
load_in_4bit = True

model_name = 'Qwen/Qwen2.5-1.5B-Instruct'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0.
   \\   /|    NVIDIA A100-PCIE-40GB. Num GPUs = 1. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
dataset = datasets.load_dataset('parquet', data_files='document-dataset.parquet')
# dataset = datasets.load_from_disk('md-version-dataset')
def formatting_prompts_func(examples):
    return { "text": [example + tokenizer.eos_token for example in examples["text"]] }

dataset = dataset['train'].map(formatting_prompts_func, batched=True)
#dataset = dataset.map(formatting_prompts_func, batched=True)

In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4387
})

In [5]:
for row in dataset[:5]['text']:
    print('------------------------')
    print(row)

------------------------
title: Policy Testing
kind: documentation
weight: 4
restrictedtoc: true
has_side_notes: true<|im_end|>
------------------------
OPA gives you a high-level declarative language
(Rego) to author fine-grained policies that
codify important requirements in your system.
To help you verify the correctness of your policies, OPA also gives you a
framework that you can use to write tests for your policies. By writing
tests for your policies you can speed up the development process of new rules
and reduce the amount of time it takes to modify rules as requirements evolve.
{}
The examples in this section try to represent the best practices. As such, they
make use of keywords that are meant to become standard keywords at some point in
time, but have been introduced gradually.
See the docs on future keywords for more information.
{}
Getting Started
Let's use an example to get started. The file below implements a simple
policy that allows new users to be created and users to

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    use_rslora=False,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # Add LoRA to all of the attention matrices
                      "gate_proj", "up_proj", "down_proj", # Add LoRA to all of the FFN matrices
                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_dropout = 0, 
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 3407,
    loftq_config = None
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [139]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    dataset_num_proc = 4,
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        num_train_epochs = 1,
        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.1,
        logging_steps = 10,
        report_to = 'none',
        seed=3407,
        output_dir = 'result'
    ),
)

In [140]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,387 | Num Epochs = 1 | Total steps = 274
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 485,212,160/5,000,000,000 (9.70% trained)


Step,Training Loss
10,2.193600
20,2.187200
30,2.031200
40,2.090600
50,2.208400
60,2.081400
70,2.096800
80,2.083300
90,2.147900
100,1.977800


In [141]:
#system_prompt = """
#You are TerraformAI, an AI agent that builds and deploys Cloud Infrastructure written in Terraform HCL. Generate a description of the Terraform program you will define, followed by a single Terraform HCL program in response to each of my Instructions. Make sure the configuration is deployable. Create IAM roles as needed. If variables are used, make sure default values are supplied. Be sure to include a valid provider configuration within a valid region. Make sure there are no undeclared resources (e.g., as references) or variables, that is, all resources and variables needed in the configuration should be fully specified.
#"""

system_prompt = """
You are TerraformAI, an AI agent that builds and deploys Cloud Infrastructure written in Terraform HCL.
"""

cot_prompt = """
Here are a few examples:

Example prompt 1: Create an AWS RDS instance (with an instance class of db.t2.micro, and don't create a final snapshot before eventual deletion) with randomly generated id and password
Example output 1: Let's think step by step. First, let's reason about the resources needed: this would be an AWS RDS instance (aws_db_instance), and resources to generate a random id and password. Second, we fill in the attributes of each resource, starting with those explicitly and implicitly mentioned in the prompt, and followed by others: for example, for the aws_db_instance, we need to set the "instance_class" attribute to "db.t2.micro", and the "skip_final_snapshot" attribute to true. Finally, we connect the resources together, as needed: here "identifier" should be connected to the "random_id" resource, and "password" should be connected to the "random_password" resource
```hcl
resource "random_id" "suffix" {
  byte_length = 4
}

resource "random_password" "db" {
  length  = 16
  special = false
}

resource "aws_db_instance" "test" {
  identifier          = "metricbeat-test-${random_id.suffix.hex}"
  allocated_storage   = 20 // Gigabytes
  engine              = "mysql"
  instance_class      = "db.t2.micro"
  db_name                = "metricbeattest"
  username            = "foo"
  password            = random_password.db.result
  skip_final_snapshot = true // Required for cleanup
}
```

Example prompt 2: Create an 20GB MySQL instance on aws with randomly generated id and password
Example output 2: Let's think step by step. First, let's reason about the resources needed: this would be an AWS RDS instance (aws_db_instance), and resources to generate a random id and password. Second, we fill in the attributes of each resource, starting with those explicitly and implicitly mentioned in the prompt, and followed by others: for example, for the aws_db_instance, we need to set the "engine" attribute to "mysql". Finally, we connect the resources together, as needed: here "identifier" should be connected to the "random_id" resource, and "password" should be connected to the "random_password" resource
```hcl
resource "random_id" "suffix" {
  byte_length = 4
}

resource "random_password" "db" {
  length  = 16
  special = false
}

resource "aws_db_instance" "test" {
  identifier          = "metricbeat-test-${random_id.suffix.hex}"
  allocated_storage   = 20 // Gigabytes
  engine              = "mysql"
  instance_class      = "db.t2.micro"
  db_name                = "metricbeattest"
  username            = "foo"
  password            = random_password.db.result
  skip_final_snapshot = true // Required for cleanup
}
```

Example prompt 3: create a AWS EFS, and create a replica of an this created EFS file system using regional storage in us-west-2
Example output 3: Let's think step by step. First, let's reason about the resources needed: this would be an AWS EFS replication resource (aws_efs_replication_configuration), and  the AWS EFS resource itself. Second, we fill in the attributes of each resource, starting with those explicitly and implicitly mentioned in the prompt, and followed by others: for example, for the aws_efs_replication_configuration, we need to set the "availability_zone_name" attribute to an availability zone that will be within the region specificed in the prompt, such as "us-west-2b". Finally, we connect the resources together, as needed: here "source_file_system_id" should be connected to the "aws_efs_file_system" resource
```hcl
resource "aws_efs_file_system" "example" {}

resource "aws_efs_replication_configuration" "example" {
  source_file_system_id = aws_efs_file_system.example.id

  destination {
    availability_zone_name = "us-west-2b"
    kms_key_id             = "1234abcd-12ab-34cd-56ef-1234567890ab"
  }
}
```

Here is the actual prompt to answer. Let's think step by step:
"""

few_shot = """
Here are a few examples:

Example prompt 1: Create an AWS RDS instance with randomly generated id and password
Example output 1: 
```hcl
resource "random_id" "suffix" {
  byte_length = 4
}

resource "random_password" "db" {
  length  = 16
  special = false
}

resource "aws_db_instance" "test" {
  identifier          = "metricbeat-test-${random_id.suffix.hex}"
  allocated_storage   = 20 // Gigabytes
  engine              = "mysql"
  instance_class      = "db.t2.micro"
  db_name                = "metricbeattest"
  username            = "foo"
  password            = random_password.db.result
  skip_final_snapshot = true // Required for cleanup
}
```

Here is the actual prompt to answer:
"""

messages = [
    #{"role": "system", "content": system_prompt},
    {"role": "user", "content": 'Create a DynamoDB Contributor Insights resource for a specific table with custom settings' }
]

In [142]:
FastLanguageModel.for_inference(model)
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
).to("cuda")

In [150]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
from threading import Thread
# thread = Thread(model.generate, {'inputs': inputs,
#     'streamer' : text_streamer,
#     'max_new_tokens' : 256,
#     'use_cache' : True,})
# thread.start()

# import textwrap
# max_print_width = 1100

# length = 0
# for j, new_text in enumerate(text_streamer):
#     if j == 0:
#         wrapped_text = textwrap.wrap(new_text, width = max_print_width)
#         length = len(wrapped_text[-1])
#         wrapped_text = "\n".join(wrapped_text)
#         print(wrapped_text, end='')
#     else:
#         length += len(new_text)
#         if length >= new_print_width:
#             length = 0
#             print()
#         print(new_text, end = '')
#     pass
# pass

model.generate(inputs=inputs, streamer=text_streamer,max_new_tokens=256,use_cache=True)

```hcl
resource "aws_dynamodb_contributor_insights" "example" 
```
The following is an example of how to configure the attributes of this resource:
| Attribute | Type | Description |
| --- | --- | --- |
| arn | string | The ARN of the AWS account that will be used as the source for contributors insights. |
| enabled | bool | Whether or not to enable contributor insights. |
| name | string | The name of the contributor insights policy. |
| partition_key_name | string | The name of the partition key in the table where the contributors insights should be applied. |
| region | string | (Optional) The region where the resources reside. If omitted, it defaults to the current region. |
| read_only | bool | Whether or not to apply the policy to all read-only endpoints. Defaults to false.<|im_end|>


tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   4021,    264,  71813,
           3506,  64724,  72037,   5101,    369,    264,   3151,   1965,    448,
           2526,   5003, 151645,    198, 151644,  77091,    198,  73594,     71,
            564,    198,   9233,    330,   8635,    814,  83348,  92581,   4831,
          34386,   2796,      1,    330,   8687,      1,    715,  13874,   3989,
            785,   2701,    374,    458,   3110,    315,   1246,    311,  14411,
            279,   8201,    315,    419,   5101,    510,     91,  16752,    760,
           3990,    760,   7662,   9248,     91,  12448,    760,  12448,    760,
          12448,   9248,     91,    796,     77,    760,    914,    760,    576,
           6261,     45,    315,    279,  23245,   2692,    429,    686,    387,
           1483,    438,    

In [ ]:
model.save_pretrained_merged("Qwen2.5-1.5B-Instruct-IaC-merged", tokenizer, save_method="merged_16bit")